In [2]:
import random
import math

In [11]:
class PrimerDesign(object):
    
    def __init__ (self, name):
        
        '''parameters for the length criterion'''
        self.max_length = 0
        self.min_length = 0
        self.penalty_length = 10
        
        '''parameters for the temperature difference criterion'''
        self.max_tdiff = 0
        self.min_tdiff = 0
        self.penalty_tdiff = 10
        
        '''parameters for the cg content criterion'''
        self.max_cg = 0
        self.min_cg = 0
        self.penalty_cg = 10
        
        '''parameters for the annealing temperature criterion'''
        self.max_temp = 0
        self.min_temp = 0
        self.penalty_temp = 10
        
        '''parameters for the run criterion'''
        self.run_threshold = 0
        self.penalty_runs = 10
        
        '''parameters for the repeat criterion'''
        self.repeat_threshold = 0
        self.penalty_repeats = 10
        
        '''parameters for the specificity criterion'''
        self.penalty_specificity = 10 
        
        '''locations where the forward primer should be chosen from'''
        self.fp_start = 100
        self.fp_end = 200
        
        '''locations where the reverse primer should be chosen from'''
        self.rp_start = 300
        self.rp_end = 400
        
        ''' parameters for the simulated annealing portion'''
        self.initial_temperature = 200
        self.stopping_temperature = 0.01
        self.drop_fraction = 0.999
        

### Task 2 

In [12]:
string = "1 gtccgggcag cccccggcgc agcgcggccg cagcagcctc cgccccccgc acggtgtgag 1 gtccgggcag cccccggcgc agcgcggccg cagcagcctc cgccccccgc acggtgtgag 1 gtccgggcag cccccggcgc agcgcggccg cagcagcctc cgccccccgc acggtgtgag 1 gtccgggcag cccccggcgc agcgcggccg cagcagcctc cgccccccgc acggtgtgag 1 gtccgggcag cccccggcgc agcgcggccg cagcagcctc cgccccccgc acggtgtgag 1 gtccgggcag cccccggcgc agcgcggccg cagcagcctc cgccccccgc acggtgtgag"

In [13]:
class PrimerDesign(PrimerDesign): 
    
    def set_dna_sequence(self, dna_sequence):
        nucleotides = ["a", "t", "c", "g"]
        output_list = []
        for char in dna_sequence: 
            if char in nucleotides: 
                output_list.append(char)
#         print("output", output_list)
        dna_sequence = "".join(output_list)
        self.dna_sequence = dna_sequence 

'gtccgggcagcccccggcgcagcgcggccgcagcagcctccgccccccgcacggtgtgaggtccgggcagcccccggcgcagcgcggccgcagcagcctccgccccccgcacggtgtgaggtccgggcagcccccggcgcagcgcggccgcagcagcctccgccccccgcacggtgtgaggtccgggcagcccccggcgcagcgcggccgcagcagcctccgccccccgcacggtgtgaggtccgggcagcccccggcgcagcgcggccgcagcagcctccgccccccgcacggtgtgaggtccgggcagcccccggcgcagcgcggccgcagcagcctccgccccccgcacggtgtgag'

### Task 3

Should I save or return the primer

In [15]:
class PrimerDesign(PrimerDesign):
    
    def func_select_random(self, sqtype='forward', length = 20 ):
        '''the length has to be a positive number'''
        
        if(sqtype == 'forward'):
#             print("forward")
            output_list = []
            start_limit = self.fp_start 
            end_limit = self.fp_end 
            start_pos = random.randint(start_limit, end_limit-length)
            forward_unprocessed = self.dna_sequence[start_pos:start_pos+length]
            for char in forward_unprocessed[::-1]:
                if char == "a":
                    output_list.append("t")
                elif char == "t":
                    output_list.append("a")
                elif char == "g":
                    output_list.append("c")
                elif char == "c":
                    output_list.append("g")
            forward_primer = "".join(output_list)
            return forward_primer 
        
        elif(sqtype == 'reverse'):
#             print("reverse")
            start_limit = self.rp_start 
            end_limit = self.rp_end
            
            start_pos= random.randint(start_limit, end_limit-length)
            reverse_primer = self.dna_sequence[start_pos: start_pos+length]
       
            return reverse_primer
        else: 
            return None
         



Test func_select_random for both forward and reverse

In [17]:
test2 = PrimerDesign("Example")
test2.set_dna_sequence(string)
test2.dna_sequence
print(test2.func_select_random(sqtype='forward', length = 20))
print(test2.func_select_random(sqtype='reverse', length=20))

gccgggggctgcccggacct
ccgggcagcccccggcgcag


### Task 4

In [19]:
class PrimerDesign(PrimerDesign): 
    
    def func_length(self, sq):
        return len(sq)
    
    def func_cg_fraction(self, sq):
        count = 0 
        for base in sq: 
            if base == 'c' or base == 'g': 
                count+=1 
        return count/len(sq)
    
    def func_temperature(self,sq):
        optimal_temp = 4*(sq.count('c')+sq.count('g'))+2*(sq.count('a')+sq.count('t'))
        return optimal_temp

In [23]:
class PrimerDesign(PrimerDesign):

    def func_count_runs(self,sq):
        count = 1 
        num = 0 
        for i in range(len(sq)):
#       Check if it is the last character

            if i == len(sq)-1:
                if count > self.run_threshold:
                    num += 1
                    count = 1 
                else:
                    if sq[i+1] == sq[i]:
                        count += 1
                    
                    else:
                        if count > self.run_threshold:
                            num += 1
                        count = 1

        return num

In [24]:
class PrimerDesign(PrimerDesign):
    def func_count_repeats(self,sq):
        
        di_repeats = ['at','ac','ag','ca','ct','cg','ga','gt','gc','ta','tc','tg']
        sequence_version1 = []
        sequence_version2= []

        for i in range(0, len(sq), 2):
            sequence_version1.append(sq[i:i+2])

        for i in range(1, len(sq), 2):
            sequence_version2.append(sq[i:i+2])

        count = 0
        count1 = 0
        total_repeats = 0
        
        for i in range(len(sequence_version1)):

            try:
                if sequence_version1[i+1] in di_repeats and sequence_version1[i+1] == sequence_version1[i]:
                count +=1
                print("match", sequence_version1[i+1], sequence_version2[i])
                
                else:
                    total_repeats += count
                    count = 0

            except:
                pass

        for i in range(len(sequence_version2)):
            try:
            
            if sequence_version2[i+1] in di_repeats and sequence_version2[i+1] == sequence_version2[i]:
                count +=1

            else:
                total_repeats += count
                count = 0
            
            except:
                pass


IndentationError: expected an indented block (<ipython-input-24-0d55f81cfa5a>, line 22)

### Task 5

In [8]:
class PrimerDesign(PrimerDesign):
    
    def cost_length(self, sq):
        '''This is given to you as an example '''
        sq_len = len(sq)
        if(sq_len > self.max_length):
            return (sq_len - self.max_length)*self.penalty_length
        elif(sq_len > self.min_length):
            return 0
        else:
            return (self.min_length - sq_len)*self.penalty_length 
    
    def cost_temperature(self, sq):
        pass 
        
    def cost_cgcontent(self,sq):
        pass
        
    def cost_temperature_difference(self, fp, rp):
        pass
    
    def cost_specificity(self, sq):
        pass
    
    def cost_runs(self, sq):
        pass
    
    def cost_repeats(self,sq):
        pass 

### Task 6

In [ ]:
class PrimerDesign(PrimerDesign):
    
    def cost_objective_function(self, fp, rp):
        '''complete the calculation of the cost'''
        cost = 0        
        return cost 
        

### Task 7

In [10]:
class PrimerDesign(PrimerDesign):
    
    def cost_objective_function_info(self, fp, rp):
        pass 

### Task 10

In [11]:
class PrimerDesign(PrimerDesign): 
    
    def func_simulated_annealing(self):
        
        temperature = self.initial_temperature
        stopping_temperature = self.stopping_temperature
        drop = self.drop_fraction
        
        pass 


### Store the DNA sequence given to you in the variable below 

In [12]:
dna_sequence = ''' '''

### Instantiate your class and read in the DNA sequence

' '

### If you need to adjust any parameter from their default values in the init method, do it here

### Show the outcome of your testing and the functions in the subsequent cells 